# CFR JSON request/response analysis

## License

Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Using the notebook

### Prerequisities

If you're not familiar with Colab notebooks, check out the
[Welcome to Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb)
notebook first for a tutorial.

To run the cells in the notebook, you need a runtime:

*   We regularly test the notebook with the public hosted runtime. This should
    be sufficient for experiments and to analyze smaller scenarios.
*   If the hosted runtime is too slow or you need to process large amount of
    data,
    [running a local runtime](https://research.google.com/colaboratory/local-runtimes.html)
    might be a better option. To use a local runtime with this notebook, start
    the local runtime with `sudo docker run -p 127.0.0.1:9000:8080 -e
    COLAB_KERNEL_MANAGER_PROXY_PORT=9000
    us-docker.pkg.dev/colab-images/public/runtime` and then follow the rest of
    the instructions from the
    [local runtime guide](https://research.google.com/colaboratory/local-runtimes.html).
    As of 2023-10-23, using the `COLAB_KERNEL_MANAGER_PROXY_PORT` option is
    needed to make file upload work correctly.

### How to use the colab

1.  Once you're connected to a runtime, the first setp is to run the cells in
    the "Imports, helper functions" section to initialize the notebook. You can
    re-run the cell "Define helper functions, ..." at any time to quickly remove
    all loaded scenarios from the notebook.

2.  Once the notebook is initialized, you will be able to add CFR scenarios and
    solutions to the notebook to analyze them. The easiest way is to upload
    either ZIP files from the fleet routing app or the scenario/solution JSON
    file pairs through the form in the section
    [Upload scenarios and solutions](#scrollTo=G6mXfeDxgA4M&line=1&uniqifier=1).

3.  Now you have data to analyze. Run any other cell in the notebook to walk
    through the data.

### Don't panic!

If you have any questions or run into issues with using the colab, contact
ondrasej at google dot com.


## Imports, helper functions (run these first)

In [ ]:
# @title Import everything (run this once)

import collections
from collections.abc import Callable, Iterable, Mapping, Sequence, Set
import dataclasses
import datetime
import functools
import glob
import io
import json
import os
import re
from typing import Any
import zipfile

from google.colab import data_table
from google.colab import files
from google.colab import output
from IPython import display
import ipywidgets
import pandas as pd

# # Clone the CFR library from GitHub, and import from there.
!git clone https://github.com/google/cfr
from cfr.python.gmpro import utils
from cfr.python.gmpro.analysis import analysis
from cfr.python.gmpro.json import cfr_json
from cfr.python.gmpro.json import human_readable
from cfr.python.gmpro.two_step_routing import two_step_routing


In [ ]:
# @title Define helper functions, reset data structures


# Increase the default limit on the number of rows in a DataTable. The default
# limit is 20k, and it is not quite enough for our use cases.
data_table.DataTable.max_rows = 250000  # @param{type: 'number'}
data_table.DataTable.num_rows_per_page = 50  # @param{type: 'number'}
data_table.DataTable.max_columns = 30  # @param{type: 'number'}

# TODO(ondrasej): Consider moving this to a separate library that can be tested
# with normal unit tests.


# The effective maximal and minimal datetime values in UTC. The `min` and `max`
# defined on the datetime class are naive datetimes and they can't be directly
# compared with the timestamps including a time zone that we use in the code.
_DATETIME_MIN_UTC = datetime.datetime.min.replace(tzinfo=datetime.timezone.utc)
_DATETIME_MAX_UTC = datetime.datetime.max.replace(tzinfo=datetime.timezone.utc)


# The scenario data used for the analyses in the notebook.
_scenarios = {}


def add_scenario_and_solution_from_bytes(
    name: str,
    scenario_bytes: bytes,
    solution_bytes: bytes,
    parking_bytes: bytes | None,
) -> analysis.Scenario:
  """Adds a scenario-solution pair to the collection of analyzed scenarios.

  Loads the scenario and solution data by treating `scenario_bytes` and
  `solution_bytes` as encoded JSON data.

  Args:
    name: The name of the new scenario. If a scenario of this name already
      exists, it is overwritten.
    scenario_bytes: The serialized JSON data of the scenario.
    solution_bytes: The serialized JSON data of the solution.
    parking_bytes: The serialized JSON data for the parking location.

  Returns:
    The added scenario.
  """
  # Lean on the automatic encoding detection in the JSON parser.
  scenario_json = json.load(io.BytesIO(scenario_bytes))
  solution_json = json.load(io.BytesIO(solution_bytes))
  parking_json = (
      json.load(io.BytesIO(parking_bytes))
      if parking_bytes is not None
      else None
  )

  scenario = analysis.Scenario(
      name=name,
      scenario=scenario_json,
      solution=solution_json,
      parking_json=parking_json,
  )
  _scenarios[name] = scenario
  return scenario


def add_scenario_and_solution_from_file(
    name: str, scenario_file: str, solution_file: str, parking_file: str | None
) -> analysis.Scenario:
  """Adds a scenario-solution pair to the collection of analyzed scenarios.

  Assumes that `scenario_file` and `solution_file` are two JSON files that are
  accessible through the local file system and that contain the data in the JSON
  format. Loads and parses the contents of the two files and adds them to the
  scenario list.

  Args:
    name: The name of the new scenario. If a scenario of this name already
      exists, it is overwritten.
    scenario_file: The file name of the scenario.
    solution_file: The file name of the solution.

  Returns:
    The added scenario.
  """
  with open(scenario_file, "rt", encoding="utf-8") as f:
    scenario_json = json.load(f)
  with open(solution_file, "rt", encoding="utf-8") as f:
    solution_json = json.load(f)
  parking_json = None
  if parking_file:  # Covers both an empty string and None.
    with open(parking_file, "rt", encoding="utf-8") as f:
      parking_json = json.load(f)
  else:
    parking_file = None
  scenario = analysis.Scenario(
      name=name,
      scenario=scenario_json,
      solution=solution_json,
      parking_json=parking_json,
  )
  _scenarios[name] = scenario
  return scenario


def all_scenarios():
  """Returns the list of all scenario names, in the lexicographic order."""
  return sorted(_scenarios.values(), key=lambda scenario: scenario.name)


def dataframe_from_all_scenarios(
    row_callback: Callable[
        [analysis.Scenario], dict[str, Any] | list[dict[str, Any]]
    ]
):
  """Creates a data frame by calling `row_callback` for each scenario.

  Expects that `row_callback` returns either a dict or a list of dicts. When it
  returns a dict, it is treated as a row in the data frame. When it returns a
  list, each element of the list is treated as a separate row.

  Creates a data frame that contains the data returned for all scenarios,
  indexed by the scenario name.
  """
  data = []
  scenarios = all_scenarios()
  index = []
  for scenario in scenarios:
    scenario_data = row_callback(scenario)
    if isinstance(scenario_data, dict):
      data.append(scenario_data)
      index.append(scenario.name)
    elif isinstance(scenario_data, list):
      data.extend(scenario_data)
      for _ in scenario_data:
        index.append(scenario.name)
  return pd.DataFrame(data, index=index)


def show_table_from_all_scenarios(
    row_callback: Callable[
        [analysis.Scenario], dict[str, Any] | list[dict[str, Any]]
    ]
):
  display.display(
      data_table.DataTable(
          dataframe_from_all_scenarios(row_callback).fillna("")
      )
  )


def get_glob_substitutions(pattern: str, filename: str) -> tuple[str]:
  """Returns substitutions to wildcards from `pattern` in `filename`.

  Assumes that `filename` is matched by the glob pattern `pattern`. Returns a
  sequence of substitutions to all wildcards in pattern that are needed to match
  `filename`. The returned sequence has one element per wildcard, and they are
  ordered by the appearance of the wildcard in the pattern.

  At this moment, only `*` is supported.

  Returns:
    A sequence that contains the substitutions.

  Raises:
    ValueError: When there is a problem with extracting the substitutions. This
      can happen when: `filename` does not match `pattern` or `pattern` contains
      unsupported wildcards.
  """
  assert len(os.path.sep) == 1
  if "?" in pattern:
    raise ValueError("'?' in the pattern is not supported.")
  if "[" in pattern:
    raise ValueError("'[' in the pattern is not supported.")
  pattern_parts = (re.escape(part) for part in pattern.split("*"))
  pattern_re = re.compile(f"([^{re.escape(os.path.sep)}]*)".join(pattern_parts))
  m = pattern_re.match(filename)
  if m is None:
    raise ValueError(f"{filename!r} does not match the pattern {pattern!r}.")
  return tuple(m.groups())


def test_get_glob_substitutions():
  assert get_glob_substitutions("foo", "foo") == ()
  assert get_glob_substitutions("*-foo", "bar-foo") == ("bar",)
  assert get_glob_substitutions("*-bar-*", "foo-bar-baz") == ("foo", "baz")


def duration_for_spreadsheet(value) -> str:
  total_seconds = int(value.total_seconds())
  hours = total_seconds // 3600
  remaining_seconds = total_seconds % 3600
  return f"{hours}:{remaining_seconds // 60:02d}:{remaining_seconds % 60:02d}"


def duration_string_for_spreadsheet(duration: cfr_json.DurationString) -> str:
  return duration_for_spreadsheet(cfr_json.parse_duration_string(duration))


test_get_glob_substitutions()

## Upload scenarios and solutions

### Upload via browser

By running the cell below, you can upload scenarios and solutions either by
uploading two JSON files, or by uploading one or more ZIP files.

When uploading JSON files, there must be exactly two of them, one for the
scenario and one for the solution. The file that has "solution" or "response" in
its name will be used as the solution file, and the file that has "scenario" or
"request" in its name will be used as the corresponding scenario file. The newly
added scenario has name `name` (which needs to be filled before uploading the
files). When the uploader fails to identify a scenario and a solution, the
upload fails.

When uploading a ZIP file, the ZIP file must be a scenario-solution zip file
compatible with the fleet routing app, i.e. it must contain two files called
"scenario.json" and "solution.json". When it also contains a file called
"parking.json", it is treated as a parking location data file as used in
[`two_step_routing_main.py`](https://github.com/google/cfr/blob/main/examples/two_step_routing/two_step_routing_main.py).
The newly added scenario has the name "{name}/{zip filename}".

In [ ]:
def upload_local_scenario_and_solution():
  name = ""  # @param {type: "string"}
  uploaded_files = files.upload()
  print()

  # If there are exactly two files, try to interpret them as a scenario/solution
  # pair. This works only if they are both JSON files and have the right names.
  if len(uploaded_files) == 2:
    scenario_file = None
    scenario_bytes = None
    solution_file = None
    solution_bytes = None
    parking_file = None
    parking_bytes = None
    # Look for a scenario and a solution.
    for filename, contents in uploaded_files.items():
      if not filename.endswith(".json"):
        continue
      if "solution" in filename or "response" in filename:
        solution_file = filename
        solution_bytes = contents
      elif "scenario" in filename or "request" in filename:
        scenario_file = filename
        scenario_bytes = contents
      elif "parking" in filename:
        parking_file = filename
        parking_bytes = contents
      else:
        print("{filename} looks neither like a scenario nor a solution.")
    if scenario_file is not None and solution_file is not None:
      scenario = add_scenario_and_solution_from_bytes(
          name,
          scenario_bytes=scenario_bytes,
          solution_bytes=solution_bytes,
          parking_bytes=parking_bytes,
      )
      print(
          f"Added scenario: {scenario.name!r} from {scenario_file} and"
          f" {solution_file}"
      )
      if parking_file is not None:
        print(f"Loaded parking data from {parking_file}.")
      return

  # Otherwise, go through all ZIP files and try to treat them as the fleet
  # routing app ZIP files.
  for filename, contents in uploaded_files.items():
    if not filename.endswith(".zip"):
      continue
    with zipfile.ZipFile(io.BytesIO(contents), mode="r") as zipped_file:
      files_in_zip = set(zipped_file.namelist())
      if (
          "solution.json" not in files_in_zip
          or "scenario.json" not in files_in_zip
      ):
        continue

      scenario_bytes = zipped_file.read("scenario.json")
      solution_bytes = zipped_file.read("solution.json")
      parking_bytes = None
      try:
        parking_bytes = zipped_file.read("parking.json")
      except KeyError:
        pass
      scenario_name = f"{name}/{filename}" if name else filename
      add_scenario_and_solution_from_bytes(
          scenario_name,
          scenario_bytes=scenario_bytes,
          solution_bytes=solution_bytes,
          parking_bytes=parking_bytes,
      )
      print(f"Added scenario {scenario_name!r} from zip {filename}")


upload_local_scenario_and_solution()

### Load multiple scenarios from a filesystem (advanced)

Multiple scenarios from the local filesystem of the runtime can be added in a
single step by using filenames with wildcards to find scenario, solution, and
optionally also parking data files. At this moment, only `*` is supported.

The loader first finds all solution JSON files by looking up all files matching
`solution_pattern`. Then, it uses the wildcard substitutions from the solution
pattern together with `name_template`, `scenario_template`, and
`parking_template` to create the name of the scenario/solution pair used in the
reports, and to find the scenario file and the parking location data file.

The templates can have two formats:

*   If the template contains the `*` character(s), then it must contain the same
    number of `*` characters as `solution_pattern`, and each `*` is replaced
    with the same substitution string as the corresponding `*` in
    `solution_pattern`.
*   If the template does not contain the `*` character, then the file name is
    created using `template.format(*substitutions)` where `substitutions` is a
    tuple of all substitutions in for `solution_pattern`.

When `parking_template` is empty, the loader assumes that the scenarios and
solutions do not have parking data and loads only the scenario and solution
files. When the scenario file or parking file (when `parking_template` is not
empty) for a given solution does not exist, then this solution is ignored.

Examples:

*   When `solution_pattern` is `*-*-solution.json`, `scenario_template` is
    `*-*-scenario.json`, and there is a file `20231003-paris-solution.json`, the
    loader will look for `20231003-paris-scenario.json` as the scenario file.
*   When `solution_pattern` is `*-*-solution-*.json`, `scenario_template` is
    `{0}-{1}-scenario.json`, and `parking_template` is `{0}-{1}-parking.json`,
    and there is a file `20231003-paris-solution-1800s.json`, the loader will
    look for `20231003-paris-scenario.json` and `20231003-paris-parking.json` as
    the scenario and parking files.

When `dry_run` is ticked, the files are matched, paired, and scenario names are
printed, but the scenarios are not loaded. Use this to test the pattern and
`name_template`. When `clean_existing` is ticked, all scenarios that were loaded
previously will be removed before loading the new ones.

In [ ]:
def _match_files_and_group_by_substitutions(
    pattern: str,
) -> Mapping[tuple[str, ...], str]:
  """Finds files matching `pattern` and groups them by wildcard substitutions."""
  files = glob.glob(pattern)
  matching_files = {}
  for filename in files:
    substitutions = get_glob_substitutions(pattern, filename)
    # The following assert should never fail if the matching algorithm is
    # deterministic.
    assert (
        substitutions not in matching_files
    ), "Multiple files with the same substitutions."
    matching_files[substitutions] = filename
  return matching_files


def _apply_substitutions(pattern, substitutions):
  num_wildcards = pattern.count("*")
  if num_wildcards > 0:
    if num_wildcards != len(substitutions):
      raise ValueError(
          "The number of substitutions in the pattern does not match the number"
          " of provided values."
      )
    filename = pattern
    for substitution in substitutions:
      filename = filename.replace("*", substitution, 1)
    return filename
  else:
    return pattern.format(*substitutions)


def add_multiple_scenarios_and_solutions():
  dry_run = False  # @param{type: "boolean"}
  clear_existing = False  # @param{type: "boolean"}
  name_template = ""  # @param{type: "string"}
  solution_pattern = ""  # @param{type: "string"}
  scenario_template = ""  # @param{type: "string"}
  parking_template = ""  # @param{type: "string"}

  solution_matches = _match_files_and_group_by_substitutions(solution_pattern)

  if not dry_run and clear_existing:
    _scenarios.clear()

  progressbar = ipywidgets.IntProgress(
      value=0,
      min=0,
      max=len(solution_matches),
      description="Loading:",
  )
  display.display(progressbar)

  new_scenarios = []
  overwritten_scenarios = []

  scenario_not_found = []
  parking_not_found = []

  for substitutions, solution_file in solution_matches.items():
    if not utils.is_non_empty_file(solution_file):
      continue
    name = _apply_substitutions(name_template, substitutions)
    scenario_file = _apply_substitutions(scenario_template, substitutions)
    if not utils.is_non_empty_file(scenario_file):
      scenario_not_found.append(name)
      continue
    parking_file = None
    if parking_template:
      parking_file = _apply_substitutions(parking_template, substitutions)
      if not utils.is_non_empty_file(parking_file):
        parking_not_found.append(name)
        continue

    if not clear_existing and name in _scenarios:
      overwritten_scenarios.append(name)
    else:
      new_scenarios.append(name)

    if not dry_run:
      add_scenario_and_solution_from_file(
          name, scenario_file, solution_file, parking_file
      )
    progressbar.value += 1

  message = []
  if scenario_not_found:
    message.append(
        "**Scenario file was not found for**: "
        + ", ".join(sorted(scenario_not_found))
    )
  if parking_not_found:
    message.append(
        "**Parking file was not found for**: "
        + ", ".join(sorted(parking_not_found))
    )

  if new_scenarios:
    message.append(
        f"**Added {len(new_scenarios)} new scenario(s)**: "
        + ", ".join(sorted(new_scenarios))
    )
  if overwritten_scenarios:
    message.append(
        f"**Overwrote {len(overwritten_scenarios)} existing scenario(s)**: "
        + ", ".join(sorted(overwritten_scenarios))
    )
  if not new_scenarios and not overwritten_scenarios:
    message.append("No scenarios were added. Check that the paths are correct.")

  display.clear_output()
  display.display(display.Markdown("\n\n".join(message)))


add_multiple_scenarios_and_solutions()

## Basic request/response data

In [ ]:
# @title Solution metrics


def get_metrics(scenario):
  # TODO(ondrasej): If metrics are not available, recompute them from route
  # details instead of taking an empty struct that will be presented as
  # all-zeros.
  metrics = scenario.solution.get("metrics", {})
  aggregated_metrics = metrics.get("aggregatedRouteMetrics", {})

  data = {
      "total cost": round(metrics.get("totalCost", 0), 2),
      "total duration": duration_string_for_spreadsheet(
          aggregated_metrics.get("totalDuration", "0s")
      ),
      "wait duration": duration_string_for_spreadsheet(
          aggregated_metrics.get("waitDuration", "0s")
      ),
      "delay duration": duration_string_for_spreadsheet(
          aggregated_metrics.get("delayDuration", "0s")
      ),
      "travel duration": duration_string_for_spreadsheet(
          aggregated_metrics.get("travelDuration", "0s")
      ),
      "travel distance meters": aggregated_metrics.get(
          "travelDistanceMeters", 0
      ),
      "skipped mandatory shipments": metrics.get(
          "skippedMandatoryShipmentCount", 0
      ),
      "used vehicles": metrics.get("usedVehicleCount", 0),
  }
  for cost_name, cost_value in metrics.get("costs", {}).items():
    readable_name = cost_name.split(".")[-1]
    data[readable_name] = round(cost_value, 2)
  return data


show_table_from_all_scenarios(get_metrics)

In [ ]:
# @title Aggregated shipment statistics


def _has_time_window(shipment):
  for delivery in shipment.get("deliveries", ()):
    if "timeWindows" in delivery:
      return True
  for delivery in shipment.get("pickups", ()):
    if "timeWindows" in delivery:
      return True
  return False


def get_basic_shipment_stats(scenario):
  include_arrival_and_departure_virtual_shipments = False  # @param {type: "boolean"}
  high_priority_shipment_threshold = None  # @param

  shipments = scenario.shipments
  filtered_shipment_index_map = {}
  if not include_arrival_and_departure_virtual_shipments:
    filtered_shipments = []
    for shipment_index, shipment in enumerate(shipments):
      label = shipment.get("label", "")
      if label.endswith(" arrival") or label.endswith(" departure"):
        continue
      filtered_shipment_index_map[shipment_index] = len(filtered_shipments)
      filtered_shipments.append(shipment)
    shipments = filtered_shipments
  else:
    filtered_shipment_index_map = {i: i for i in range(len(shipments))}

  # Filtering shipments may have changed shipment indices. Whenever we work with
  # a shipment index from the unfiltered (raw) request/response, we need to run
  # them through filtered_shipment_index_map.
  def filtered_indices(indices: Iterable[int]) -> Set[int]:
    filtered_indices = set()
    for index in indices:
      filtered_index = filtered_shipment_index_map.get(index)
      if filtered_index is not None:
        filtered_indices.add(filtered_index)
    return filtered_indices

  skipped_shipments = scenario.skipped_shipments
  skipped_shipment_indices = filtered_indices(
      skipped_shipment.get("index", 0) for skipped_shipment in skipped_shipments
  )

  num_cfr_shipments_from_parking = 0
  num_actual_shipments_from_parking = 0

  if scenario.parking_for_shipment is not None:
    num_cfr_shipments_from_parking = len(scenario.parking_for_shipment)
    for raw_shipment_index in scenario.parking_for_shipment:
      shipment_index = filtered_shipment_index_map.get(raw_shipment_index)
      if shipment_index is None:
        continue
      shipment = shipments[shipment_index]
      num_actual_shipments_from_parking += (
          shipment.get("label", "").count(",") + 1
      )

  shipments_with_time_window = [
      shipment for shipment in shipments if _has_time_window(shipment)
  ]
  skipped_shipments_with_time_window = [
      shipment
      for shipment_index, shipment in enumerate(shipments)
      if _has_time_window(shipment)
      and shipment_index in skipped_shipment_indices
  ]
  if high_priority_shipment_threshold is not None:
    high_priority_shipments = [
        shipment
        for shipment in shipments
        if shipment.get("penaltyCost", 0) >= high_priority_shipment_threshold
    ]
    skipped_high_priority = [
        shipment
        for shipment_index, shipment in enumerate(shipments)
        if shipment_index in skipped_shipment_indices
        and shipment.get("penaltyCost", 0) >= high_priority_shipment_threshold
    ]
  shipment_stats = {
      "# CFR shipments": len(shipments),
      "# actual shipments": sum(
          shipment["label"].count(",") + 1 for shipment in shipments
      ),
  }
  if high_priority_shipment_threshold is not None:
    shipment_stats["# high priority CFR shipments"] = len(
        high_priority_shipments
    )
    shipment_stats["# high priority actual shipments"] = sum(
        shipment["label"].count(",") + 1 for shipment in high_priority_shipments
    )

  shipment_stats["# CFR shipments w/TW"] = len(shipments_with_time_window)
  shipment_stats["# actual shipments w/TW"] = sum(
      shipment["label"].count(",") + 1
      for shipment in shipments_with_time_window
  )
  shipment_stats["# skipped CFR shipments"] = len(skipped_shipments)
  shipment_stats["# skipped actual shipments"] = sum(
      shipment["label"].count(",") + 1 for shipment in skipped_shipments
  )
  if high_priority_shipment_threshold is not None:
    shipment_stats["# skipped high priority CFR shipments"] = len(
        skipped_high_priority
    )
    shipment_stats["# skipped high priority actual shipments"] = sum(
        shipment["label"].count(",") + 1 for shipment in skipped_high_priority
    )

  shipment_stats["# skipped CFR shipments w/TW"] = len(
      skipped_shipments_with_time_window
  )
  shipment_stats["# skipped actual shipments w/TW"] = sum(
      shipment["label"].count(",") + 1
      for shipment in skipped_shipments_with_time_window
  )
  shipment_stats["# CFR shipments from parking"] = (
      num_cfr_shipments_from_parking
  )
  shipment_stats["# actual shipments from parking"] = (
      num_actual_shipments_from_parking
  )
  shipment_stats["penalty cost"] = sum(
      shipments[shipment_index].get("penaltyCost", 0)
      for shipment_index in skipped_shipment_indices
  )

  return shipment_stats


show_table_from_all_scenarios(get_basic_shipment_stats)

In [ ]:
# @title Aggregated vehicle statistics


def _safe_percentage(value, maximum) -> str:
  if not maximum:
    # The maximum is zero or a zero-like value (e.g. datetime.timedelta of zero
    # duration).
    return ""
  return f"{100 * value / maximum:.2f} %"


def get_basic_vehicle_stats(scenario):
  uturn_threshold_degrees = 170  # @param{type: "number"}

  warp_threshold_meters = 0  # @param{type: "number"}

  vehicles = scenario.vehicles
  routes = scenario.routes

  max_working_hours = datetime.timedelta()
  soft_working_hours = datetime.timedelta()
  actual_working_hours = datetime.timedelta()
  wait_hours = datetime.timedelta()
  negative_wait_hours = datetime.timedelta()
  travel_hours = datetime.timedelta()
  travel_distance_meters = 0

  num_uturns = 0

  num_warp_points = 0
  warp_distance_meters = 0

  num_time_travel = 0
  num_hard_time_travel = 0
  for vehicle, route in zip(vehicles, routes, strict=True):
    route_metrics = route.get("metrics", {})

    max_working_hours += cfr_json.get_vehicle_max_working_hours(
        scenario.model, vehicle
    )
    soft_working_hours += cfr_json.get_vehicle_max_working_hours(
        scenario.model, vehicle, soft_limit=True
    )
    actual_working_hours += cfr_json.get_vehicle_actual_working_hours(route)
    num_time_travel += cfr_json.get_num_decreasing_visit_times(
        scenario.model, route, consider_visit_duration=True
    )
    num_hard_time_travel += cfr_json.get_num_decreasing_visit_times(
        scenario.model, route, consider_visit_duration=False
    )
    wait_hours += max(
        datetime.timedelta(), analysis.get_vehicle_wait_hours(route)
    )
    negative_wait_hours += analysis.get_vehicle_negative_wait_hours(route)
    travel_hours += analysis.get_vehicle_travel_hours(route)
    travel_distance_meters += route_metrics.get("travelDistanceMeters", 0)

    uturns = analysis.get_visit_turn_angles(
        scenario.model, route, threshold_angle_degrees=uturn_threshold_degrees
    )
    num_uturns += sum(1 for _ in uturns)

    for warp_point in analysis.get_visit_warp_distances(
        scenario.model, route, threshold_meters=warp_threshold_meters
    ):
      num_warp_points += 1
      warp_distance_meters += warp_point.warp_distance_meters

  return {
      "# vehicles": len(vehicles),
      "max working time": duration_for_spreadsheet(max_working_hours),
      "soft working time": duration_for_spreadsheet(soft_working_hours),
      "actual working time": duration_for_spreadsheet(actual_working_hours),
      "wait time": duration_for_spreadsheet(wait_hours),
      "travel time": duration_for_spreadsheet(travel_hours),
      "travel distance meters": travel_distance_meters,
      "max working time %": _safe_percentage(
          actual_working_hours, max_working_hours
      ),
      "soft working time %": _safe_percentage(
          actual_working_hours, soft_working_hours
      ),
      "wait time %": _safe_percentage(wait_hours, actual_working_hours),
      "# soft time travel": str(num_time_travel),
      "negative wait time": duration_for_spreadsheet(negative_wait_hours),
      "# time travel": str(num_hard_time_travel),
      "# uturns": num_uturns,
      "# warp points": num_warp_points,
      "# warp distance meters": warp_distance_meters,
  }


show_table_from_all_scenarios(get_basic_vehicle_stats)

In [ ]:
# @title Shipment list


def get_shipment_list(scenario):
  shipments = scenario.shipments

  shipment_visits = collections.defaultdict(list)
  shipment_to_vehicle = {}
  for route_index, route in enumerate(scenario.routes):
    for visit_index, visit in enumerate(route.get("visits", ())):
      shipment_index = visit.get("shipmentIndex", 0)
      shipment_to_vehicle[shipment_index] = route_index
      shipment_visits[shipment_index].append(visit)

  data = []
  for shipment_index, shipment in enumerate(shipments):
    pickup_time_windows = []
    pickup_durations = []
    pickups = shipment.get("pickups", ())
    for pickup in pickups:
      if (time_windows := pickup.get("timeWindows")) is not None:
        pickup_time_windows.append(human_readable.time_windows(time_windows))
      pickup_durations.append(pickup.get("duration", "0s"))
    delivery_time_windows = []
    delivery_durations = []
    deliveries = shipment.get("deliveries", ())
    for delivery in deliveries:
      if (time_windows := delivery.get("timeWindows")) is not None:
        delivery_time_windows.append(human_readable.time_windows(time_windows))
      delivery_durations.append(delivery.get("duration", "0s"))
    vehicle_index = shipment_to_vehicle.get(shipment_index)
    vehicle_label = (
        f"{vehicle_index}: {scenario.vehicle_label(vehicle_index)}"
        if vehicle_index is not None
        else ""
    )
    # Each shipment has at most two visits: a pickup and a delivery.
    pickup_time = ""
    delivery_time = ""
    for visit in shipment_visits[shipment_index]:
      if visit.get("isPickup"):
        pickup_time = visit["startTime"]
      else:
        delivery_time = visit["startTime"]
    data.append({
        "shipment_index": shipment_index,
        "label": shipment.get("label", ""),
        "vehicle": vehicle_label,
        "pickup time": pickup_time,
        "delivery time": delivery_time,
        "type": shipment.get("shipmentType", ""),
        "penalty": shipment.get("penaltyCost", -1),
        "pickup time windows": " | ".join(pickup_time_windows),
        "pickup durations": " | ".join(pickup_durations),
        "delivery time windows": " | ".join(delivery_time_windows),
        "delivery durations": " | ".join(delivery_durations),
        "allowed vehicles": ", ".join(
            scenario.vehicle_labels(shipment.get("allowedVehicleIndices", ()))
        ),
        "parking tag": scenario.parking_for_shipment.get(shipment_index, ""),
    })
  return data


show_table_from_all_scenarios(get_shipment_list)

In [ ]:
# @title Vehicle list


def get_vehicle_list(scenario):
  used_vehicles_only = False  # @param{type: "boolean"}
  split_by_breaks = True  # @param{type: "boolean"}
  uturn_threshod_degrees = 170  # @param{type: "number"}
  warp_threshold_meters = 50  # @param{type: "number"}
  show_duration_statistics = False  # @param{type: "boolean"}
  show_traffic_statistics = False  # @param{type: "boolean"}
  show_parking_statistics = False  # @param{type: "boolean"}
  parking_data = scenario.parking_location_data
  shipments = scenario.shipments
  data = []

  for vehicle_index, route in enumerate(scenario.routes):
    vehicle = scenario.vehicles[vehicle_index]
    route_metrics = route.get("metrics", {})
    row = {
        "vehicle_index": vehicle_index,
        "label": scenario.vehicle_label(vehicle_index),
        "# shipments": "",
        "travel distance meters": route_metrics.get("travelDistanceMeters", 0),
        "# u-turns": "",
        "# warp points": "",
        "warp distance (m)": "",
    }
    if show_duration_statistics:
      row["duration"] = ""
      row["first to last visit"] = ""
      row["start time"] = ""
      row["first visit"] = ""
      row["90% shipments delivered"] = ""
      row["last visit"] = ""
      row["end time"] = ""
      row["max working time"] = ""
      row["soft working time"] = ""
      row["actual working time"] = ""
      row["wait time"] = ""
    if show_traffic_statistics:
      row["negative wait time"] = ""
      row["# time travel"] = ""
      row["# soft time travel"] = ""
    if show_parking_statistics:
      row["# ping-pongs"] = ""
      row["# bad ping-pongs"] = ""
      row["bad ping-pong tags"] = ""
      row["# sandwiches"] = ""
      row["# bad sandwiches"] = ""
      row["bad sandwich tags"] = ""
      row["# sandwiches (v1)"] = ""
      row["# bad sandwiches (v1)"] = ""
      row["bad sandwich tags (v1)"] = ""

    if show_duration_statistics:
      _set_if_enabled(
          row,
          "max working time",
          cfr_json.get_vehicle_max_working_hours(scenario.model, vehicle),
      )
      _set_if_enabled(
          row,
          "soft working time",
          cfr_json.get_vehicle_max_working_hours(
              scenario.model, vehicle, soft_limit=True
          ),
      )

    visits = route.get("visits", ())
    if not visits:
      # Unused vehicle. Nothing to see here.
      if not used_vehicles_only:
        data.append(row)
      continue

    num_ping_pongs, bad_ping_pong_tags = analysis.get_num_ping_pongs(
        scenario, vehicle_index, split_by_breaks=split_by_breaks
    )
    num_sandwiches_v1, bad_sandwich_tags_v1 = analysis.get_num_sandwiches(
        scenario, vehicle_index
    )
    num_sandwiches, bad_sandwich_tags = analysis.analyse_bad_sandwiches(
        scenario, vehicle_index
    )

    num_time_travel = cfr_json.get_num_decreasing_visit_times(
        scenario.model, route, False
    )
    num_soft_time_travel = cfr_json.get_num_decreasing_visit_times(
        scenario.model, route, True
    )

    start_time = cfr_json.parse_time_string(route["vehicleStartTime"])
    end_time = cfr_json.parse_time_string(route["vehicleEndTime"])

    consecutive_visits = parking_data.consecutive_visits.get(vehicle_index)
    non_consecutive_visits = parking_data.non_consecutive_visits.get(
        vehicle_index
    )

    first_visit_start = cfr_json.parse_time_string(visits[0]["startTime"])
    last_visit_start = cfr_json.parse_time_string(visits[-1]["startTime"])

    _, shipments_90p_time = analysis.get_percentile_visit_time(
        scenario.model, route, 90, False
    )

    num_uturns = 0
    for _ in analysis.get_visit_turn_angles(
        scenario.model, route, threshold_angle_degrees=uturn_threshod_degrees
    ):
      num_uturns += 1

    num_warp_points = 0
    total_warp_distance = 0
    for warp_point in analysis.get_visit_warp_distances(
        scenario.model, route, threshold_meters=warp_threshold_meters
    ):
      num_warp_points += 1
      total_warp_distance += warp_point.warp_distance_meters

    row["# shipments"] = len(visits)
    if show_duration_statistics:
      row["duration"] = str(end_time - start_time)
      row["first to last visit"] = str(last_visit_start - first_visit_start)
      row["start time"] = str(start_time)
      row["first visit"] = str(first_visit_start)
      row["90% shipments delivered"] = str(shipments_90p_time)
      row["last visit"] = str(last_visit_start)
      row["end time"] = str(end_time)
      row["max end time"] = cfr_json.get_vehicle_latest_end(
          scenario.model, vehicle
      )
      row["max working time"] = str(
          cfr_json.get_vehicle_max_working_hours(scenario.model, vehicle)
      )
      row["soft working time"] = str(
          cfr_json.get_vehicle_max_working_hours(
              scenario.model, vehicle, soft_limit=True
          )
      )
      row["actual working time"] = str(
          cfr_json.get_vehicle_actual_working_hours(route)
      )
      row["wait time"] = str(
          max(datetime.timedelta(), analysis.get_vehicle_wait_hours(route))
      )
    if show_traffic_statistics:
      row["negative wait time"] = duration_for_spreadsheet(
          analysis.get_vehicle_negative_wait_hours(route)
      )
      row["# time travel"] = str(num_time_travel)
      row["# soft time travel"] = str(num_soft_time_travel)
    if show_parking_statistics:
      row["# ping-pongs"] = str(num_ping_pongs)
      row["# bad ping-pongs"] = len(bad_ping_pong_tags)
      row["bad ping-pong tags"] = ", ".join(bad_ping_pong_tags)
      row["# sandwiches"] = str(num_sandwiches)
      row["# bad sandwiches"] = len(bad_sandwich_tags)
      row["bad sandwich tags"] = ", ".join(bad_sandwich_tags)
      row["# sandwiches (v1)"] = str(num_sandwiches_v1)
      row["# bad sandwiches (v1)"] = len(bad_sandwich_tags_v1)
      row["bad sandwich tags (v1)"] = ", ".join(bad_sandwich_tags_v1)
    row["# u-turns"] = num_uturns
    row["# warp points"] = num_warp_points
    row["warp distance (m)"] = total_warp_distance

    data.append(row)
  return data


show_table_from_all_scenarios(get_vehicle_list)

In [ ]:
# @title Skipped shipments


def get_skipped_shipment_list(scenario):
  data = []
  shipments = scenario.shipments
  for skipped_shipment in scenario.skipped_shipments:
    skipped_shipment_index = skipped_shipment.get("index", 0)
    shipment = shipments[skipped_shipment_index]
    allowed_vehicles = scenario.vehicle_labels(
        shipment.get("allowedVehicleIndices", ())
    )
    data.append({
        "shipment index": skipped_shipment_index,
        "label": skipped_shipment.get("label", ""),
        "allowed vehicles": allowed_vehicles,
        "type": shipment.get("shipmentType", ""),
        "reasons": str(skipped_shipment.get("reasons", "")),
        "penalty cost": shipment.get("penaltyCost", -1),
    })
  return data


show_table_from_all_scenarios(get_skipped_shipment_list)

In [ ]:
# @title Shipments by time window


def get_num_shipments_by_time_window(scenario):
  data = []
  shipments_by_time_window = collections.defaultdict(list)
  skipped_shipments_by_time_window = collections.defaultdict(list)
  # TODO(ondrasej): Make this work for pickups too.
  for shipment_index, shipment in enumerate(scenario.shipments):
    allowed_vehicles = ", ".join(
        scenario.vehicle_labels(shipment.get("allowedVehicleIndices", ()))
    )
    for delivery in shipment.get("deliveries", ()):
      time_windows = delivery.get("timeWindows", ())
      str_time_window = human_readable.time_windows(time_windows)
      shipments_by_time_window[allowed_vehicles, str_time_window].append(
          shipment
      )
      if shipment_index in scenario.skipped_shipment_indices:
        skipped_shipments_by_time_window[
            allowed_vehicles, str_time_window
        ].append(shipment)

  for (allowed_vehicles, time_window), shipments in sorted(
      shipments_by_time_window.items()
  ):
    skipped_shipments = skipped_shipments_by_time_window.get(
        (allowed_vehicles, time_window), ()
    )
    data.append({
        "allowed vehicles": allowed_vehicles,
        "time window": time_window or "(none)",
        "# CFR shipments": len(shipments),
        "# actual shipments": sum(
            shipment.get("label", "").count(",") + 1 for shipment in shipments
        ),
        "# skipped CFR shipments": len(skipped_shipments),
        "# skipped actual shipments": sum(
            shipment.get("label", "").count(",") + 1
            for shipment in skipped_shipments
        ),
    })

  return data


show_table_from_all_scenarios(get_num_shipments_by_time_window)

## Vehicle-shipment grouping

In [ ]:
# @title Shipments by allowed vehicles
#
# @markdown Groups shipments by the vehicles that can handle them as expressed
# @markdown in the `Shipment.allowedVehicleIndices` field. Each shipment is
# @markdown counted exactly once (appears in one row); each vehicle may appear
# @markdown in zero, one or more rows.
# @markdown
# @markdown - _# shipments_ is the total number of shipments that can be handled
# @markdown   by exactly these vehicles.
# @markdown - _# skipped shipments_ is the number of skipped shipments in the
# @markdown   solution that can be handled only by this group of vehicles.


def get_vehicle_shipment_groups(scenario):
  groups = sorted(analysis.get_vehicle_shipment_groups(scenario.model))
  data = []
  for vehicle_indices, shipment_indices in groups:
    skipped_shipments_in_group = (
        scenario.skipped_shipment_indices & shipment_indices
    )
    data.append({
        "allowed vehicles": ", ".join(scenario.vehicle_labels(vehicle_indices)),
        "# shipments": len(shipment_indices),
        "# skipped shipments": len(skipped_shipments_in_group),
    })

  # Sort the list by vehicle labels to make it easier to read.
  data.sort(key=lambda x: x["allowed vehicles"])
  return data


show_table_from_all_scenarios(get_vehicle_shipment_groups)

## Detailed vehicle data

In [ ]:
# @title Individual vehicle route


def _get_vehicle_route_rows(scenario, vehicle_index):
  model = scenario.scenario["model"]

  shipments = scenario.shipments
  vehicles = scenario.vehicles

  route = scenario.routes[vehicle_index]
  vehicle = vehicles[vehicle_index]
  vehicle_label = vehicle.get("label", "")

  visits = route.get("visits", ())
  if not visits:
    output.clear(output_tags=["table_container"])
    with output.use_tags("table_container", append=True):
      print("The route is empty")
    return ()

  data = []
  transitions = route.get("transitions", ())
  start_time = cfr_json.parse_time_string(route["vehicleStartTime"])

  # We'll be popping elements from the list, so we need to make a copy to
  # avoid destroying the data.
  breaks = list(route.get("breaks", ()))

  # If there are no breaks, we use datetime.max as the time of the next break
  # so that the case of no (more) breaks doesn't need any special handling
  # in add_breaks_before_timestamp().
  def get_next_break_start():
    return (
        cfr_json.parse_time_string(breaks[0]["startTime"])
        if breaks
        else _DATETIME_MAX_UTC
    )

  next_break_start = get_next_break_start()

  def add_breaks_before_timestamp(timestamp):
    nonlocal next_break_start
    while next_break_start < timestamp:
      current_break = breaks.pop(0)
      data.append({
          "vehicle": vehicle_label,
          "shipment index": "",
          "shipment": "",
          "shipment type": "",
          "penalty cost": "",
          "type": "break",
          "latlng": "",
          "start": str(next_break_start),
          "since start": str(next_break_start - start_time),
          "duration": current_break.get("duration", "0s"),
          "distance": "",
          "transition": "",
          "time window": "",
      })
      next_break_start = get_next_break_start()

  add_breaks_before_timestamp(start_time)
  data.append({
      "vehicle": vehicle_label,
      "shipment index": "",
      "shipment": "start",
      "shipment type": "",
      "penalty cost": "",
      "type": "",
      "latlng": human_readable.vehicle_start_location(vehicle),
      "start": str(start_time),
      "since start": "0:00:00",
      "duration": "0s",
      "distance": f"{int(transitions[0].get('travelDistanceMeters', 0.0))} m",
      "transition": (
          human_readable.transition_duration(transitions[0])
          if transitions
          else ""
      ),
      "time window": human_readable.time_windows(
          vehicle.get("startTimeWindows")
      ),
  })

  turn_angles = {
      turn.visit_index: turn.angle_degrees
      for turn in analysis.get_visit_turn_angles(scenario.model, route, -1)
  }
  warp_distances = {
      warp_point.visit_index: warp_point.warp_distance_meters
      for warp_point in analysis.get_visit_warp_distances(
          scenario.model, route, -1
      )
  }
  for visit_index, visit in enumerate(visits):
    visit_request = cfr_json.get_visit_request(scenario.model, visit)
    transition_out = transitions[visit_index + 1]
    shipment_index = visit.get("shipmentIndex", 0)
    shipment = shipments[shipment_index]
    visit_start = cfr_json.parse_time_string(visit["startTime"])
    visit_type = "pickup" if visit.get("isPickup") else "delivery"

    turn_angle = turn_angles.get(visit_index)
    turn_angle = round(turn_angle, 2) if turn_angle is not None else None

    warp_distance = warp_distances.get(visit_index)
    warp_distance = (
        round(warp_distance, 2) if warp_distance is not None else None
    )

    add_breaks_before_timestamp(visit_start)
    data.append({
        "vehicle": vehicle_label,
        "shipment index": visit.get("shipmentIndex", 0),
        "shipment": visit.get("shipmentLabel", ""),
        "shipment type": shipment.get("shipmentType", ""),
        "penalty cost": shipment.get("penaltyCost", -1),
        "type": visit_type,
        "latlng": human_readable.visit_request_location(visit_request),
        "start": str(visit_start),
        "since start": str(visit_start - start_time),
        "duration": visit_request.get("duration"),
        "distance": f"{int(transition_out.get('travelDistanceMeters', 0.0))} m",
        "transition": human_readable.transition_duration(transition_out),
        "time window": human_readable.time_windows(
            visit_request.get("timeWindows")
        ),
        "turn angle": turn_angle,
        "warp meters": warp_distance,
    })
  end_time = cfr_json.parse_time_string(route["vehicleEndTime"])

  add_breaks_before_timestamp(end_time)
  data.append({
      "vehicle": vehicle_label,
      "shipment index": "",
      "shipment": "end",
      "type": "",
      "latlng": human_readable.vehicle_end_location(vehicle),
      "start": str(end_time),
      "since start": str(end_time - start_time),
      "duration": "0s",
      "transition": "",
      "time window": vehicle.get("endTimeWindows", ""),
  })
  add_breaks_before_timestamp(_DATETIME_MAX_UTC)
  return data


def create_individual_vehicle_route_ui():
  scenario_selector = ipywidgets.Dropdown(description="Scenario:", options=[])
  vehicle_selector = ipywidgets.Dropdown(description="Vehicle:", options=[])
  refresh_button = ipywidgets.Button(description="Refresh")

  display.display(
      ipywidgets.HBox(
          children=[scenario_selector, vehicle_selector, refresh_button]
      )
  )

  scenario = None

  def on_scenario_selected(*kwargs):
    nonlocal scenario
    scenario = _scenarios[scenario_selector.value]
    vehicle_selector_options = ["(all)"]
    vehicle_selector_options.extend(
        f"{vehicle_index}: {vehicle.get('label', '')}"
        for vehicle_index, vehicle in enumerate(scenario.vehicles)
    )
    vehicle_selector.options = vehicle_selector_options

  scenario_selector.observe(on_scenario_selected, names=["value"])

  def on_refresh(_):
    if vehicle_selector.value == "(all)":
      data = []
      for vehicle_index in range(len(scenario.vehicles)):
        data.extend(_get_vehicle_route_rows(scenario, vehicle_index))
    else:
      vehicle_index = int(vehicle_selector.value.split(":")[0])
      data = _get_vehicle_route_rows(scenario, vehicle_index)

    output.clear(output_tags=["table_container"])
    with output.use_tags("table_container", append=True):
      display.display(data_table.DataTable(pd.DataFrame(data)))

  refresh_button.on_click(on_refresh)
  scenario_selector.options = sorted(_scenarios.keys())


create_individual_vehicle_route_ui()

## Parking location data

### Definitions

*   **Parking ping-pong** is the situation where the same parking location is
    visited by a vehicle multiple times in a row. In practice, this corresponds
    to multiple delivery rounds from this parking and it can happen when the
    number of shipments delivered from this parking location exceeds the
    delivery capacity of this parking. This may also happen as an artifact of
    the planning algorithm.

    We say that a ping-pong is a **bad ping-pong** when the number of delivery
    rounds in the ping-pong is higher than what would be required by the load
    limits while delivering from the parking. For example, when there are 50
    shipments delivered from parking P123 and the load limit for deliveries from
    this parking is 20, a plan where the shipments are delivered in 4 or more
    rounds scheduled back to back has a case of bad ping-pong. If the shipments
    are delivered in 3 rounds scheduled back to back, this is not a case of bad
    ping-pong.

*   **Parking sandwich** is the situation where the same parking location is
    visited multiple times by the same vehicle, but the vehicle leaves the
    parking location between the visits. For example, when the vehicle delivers
    shipments from parking P123, then from parking P456, and then again from
    P123, it is a case of parking sandwich.

    Parking sandwiches may appear on a route when shipments delivered from a
    parking have time windows that are far apart, but they may also appear
    without time windows, for example when there are no costs making the solver
    group visits to the same parking.

    We say that a sandwich is a **bad sandwich** when the separated visits to
    the parking location are because of other reasons than time windows that are
    far apart.

*   **Parking party** is the situation where multiple vehicles visit the same
    parking location. A parking party may happen when there are more shipments
    delivered from the parking location than a single vehicle can handle. It may
    also happen when the costs and constraints of a model are set up in such a
    way that using multiple vehicles leads to a lower cost.

    We say that a parking party is a **bad party** when the party happens
    because of reasons other than vehicle/delivery capacity constraints. Note
    that a party may also be created when there are too many shipments with the
    same time window than a single vehicle can deliver. Such a case is also not
    considered a bad party.

In [ ]:
# @title Parking location aggregated statistics


def get_parking_locations_summary(scenario):
  split_by_breaks = True  # @param {type: "boolean"}
  buffer_time_seconds = 3600  # @param {type: "number"}
  buffer_time = datetime.timedelta(seconds=buffer_time_seconds)
  parking_data = scenario.parking_location_data

  party_stats = analysis.get_parking_party_stats(
      scenario, buffer_time=buffer_time
  )

  num_ping_pongs = 0
  num_bad_ping_pongs = 0
  num_sandwiches = 0
  num_bad_sandwiches = 0
  num_sandwiches_v1 = 0
  num_bad_sandwiches_v1 = 0
  for vehicle_index in range(len(scenario.routes)):
    num_vehicle_ping_pongs, bad_ping_pong_tags = analysis.get_num_ping_pongs(
        scenario, vehicle_index, split_by_breaks
    )
    num_ping_pongs += num_vehicle_ping_pongs
    num_bad_ping_pongs += len(bad_ping_pong_tags)

    num_vehicle_sandwiches, bad_sandwich_tags = analysis.analyse_bad_sandwiches(
        scenario, vehicle_index
    )
    num_sandwiches += num_vehicle_sandwiches
    num_bad_sandwiches += len(bad_sandwich_tags)

    num_vehicle_sandwiches_v1, bad_sandwich_tags_v1 = (
        analysis.get_num_sandwiches(scenario, vehicle_index)
    )
    num_sandwiches_v1 += num_vehicle_sandwiches_v1
    num_bad_sandwiches_v1 += len(bad_sandwich_tags_v1)

  assert num_sandwiches == num_sandwiches_v1, (
      "Different versions of parking sandwich analysis return a different"
      " number of all parking sandwiches."
  )

  return [{
      "# distinct parkings": len(scenario.parking_locations),
      "# distinct visited parkings": len(parking_data.all_parking_tags),
      "# visits to parking": parking_data.num_all_visits_to_parking,
      "# parkings served by multiple vehicles": (
          party_stats.num_parkings_with_multiple_vehicles
      ),
      "# party visits": party_stats.num_party_visits,
      "# overlapping party visits": party_stats.num_overlapping_visit_pairs,
      "max overlapping party visits": (
          party_stats.max_vehicles_at_parking_at_once
      ),
      "# ping-pong visits": sum(
          len(visits) for visits in parking_data.consecutive_visits.values()
      ),
      "# ping-pongs": num_ping_pongs,
      "# bad ping-pongs": num_bad_ping_pongs,
      "# sandwich visits": sum(
          len(visits) for visits in parking_data.non_consecutive_visits.values()
      ),
      "# sandwiches": num_sandwiches,
      "# bad sandwiches": num_bad_sandwiches,
      "# bad sandwiches (v1)": num_bad_sandwiches_v1,
  }]


show_table_from_all_scenarios(get_parking_locations_summary)

In [ ]:
# @title Parking location list


def _merge_overlaps(overlaps):
  """Merges back-to-back overlaps by the same vehicle.

  Parking ping-pongs may create multiple overlaps that are back-to-back and that
  use the same vehicles. This is hard to read for the user and does not change
  the fact that there are multiple vehicles in the parking, so we just merge
  those into a single overlap.
  """
  previous_vehicles = ()
  previous_tag = None
  previous_end_time = None
  start_time = None
  for overlap in overlaps:
    if start_time is None:
      start_time = overlap.start_time
    elif (
        previous_vehicles != overlap.vehicles
        or previous_tag != overlap.parking_tag
        or previous_end_time != overlap.start_time
    ):
      yield analysis.OverlappingParkingVisit(
          parking_tag=previous_tag,
          start_time=start_time,
          vehicles=previous_vehicles,
          end_time=previous_end_time,
      )
      start_time = overlap.start_time
    previous_vehicles = overlap.vehicles
    previous_tag = overlap.parking_tag
    previous_end_time = overlap.end_time

  if start_time is not None:
    yield analysis.OverlappingParkingVisit(
        parking_tag=previous_tag,
        start_time=start_time,
        end_time=previous_end_time,
        vehicles=previous_vehicles,
    )


def get_parking_location_list(scenario):
  parking_data = scenario.parking_location_data
  party_stats = analysis.get_parking_party_stats(
      scenario, datetime.timedelta(0)
  )
  vehicles = scenario.vehicles
  data = []
  shipments = scenario.shipments

  overlaps_by_parking = collections.defaultdict(list)
  for overlap in party_stats.overlapping_visits:
    overlaps_by_parking[overlap.parking_tag].append(overlap)

  for parking_tag in sorted(scenario.parking_locations):
    shipments_for_parking = scenario.shipments_for_parking.get(parking_tag, ())
    parking_vehicles = parking_data.vehicles_by_parking.get(parking_tag, ())
    skipped_shipments_for_parking = set(
        shipment_index
        for shipment_index in shipments_for_parking
        if shipment_index in scenario.skipped_shipment_indices
    )

    num_cfr_shipments = len(shipments_for_parking)
    num_actual_shipments = sum(
        shipments[shipment].get("label", "").count(",") + 1
        for shipment in shipments_for_parking
    )

    num_skipped_cfr_shipments = len(skipped_shipments_for_parking)
    num_skipped_actual_shipments = sum(
        shipments[shipment].get("label", "").count(",") + 1
        for shipment in skipped_shipments_for_parking
    )

    vehicle_labels = sorted(
        scenario.vehicle_label(vehicle_index)
        for vehicle_index in parking_vehicles
    )

    overlaps = overlaps_by_parking.get(parking_tag, [])
    overlaps.sort(key=lambda overlap: overlap.start_time)
    overlap_parts = []
    for overlap in _merge_overlaps(overlaps):
      vehicles = ", ".join(
          scenario.vehicle_label(vehicle_index)
          for vehicle_index in overlap.vehicles
      )
      overlap_parts.append(
          f"{overlap.start_time} - {overlap.end_time}: {vehicles}"
      )

    data.append({
        "parking_tag": parking_tag,
        "# visits": parking_data.num_visits_to_parking[parking_tag],
        "# CFR shipments": num_cfr_shipments,
        "# actual shipments": num_actual_shipments,
        "# skipped CFR shipments": num_skipped_cfr_shipments,
        "# skipped actual shipments": num_skipped_actual_shipments,
        "# vehicles": len(parking_vehicles),
        "vehicles": ", ".join(vehicle_labels),
        "overlaps": " | ".join(overlap_parts),
    })
  return data


show_table_from_all_scenarios(get_parking_location_list)

In [ ]:
# @title Shipments delivered from a parking location


def _shipment_row(scenario, shipment_index):
  shipment = scenario.shipments[shipment_index]
  time_windows = []
  for delivery in shipment.get("deliveries", ()):
    time_windows.extend(delivery.get("timeWindows", ()))
  vehicle_label = (
      ""
      if shipment_index in scenario.skipped_shipment_indices
      else scenario.vehicle_label(scenario.vehicle_for_shipment[shipment_index])
  )
  shipment_row = {
      "label": shipment.get("label", ""),
      "allowed vehicles": ", ".join(
          str(vehicle) for vehicle in shipment.get("allowedVehicleIndices", ())
      ),
      "delivery time window": human_readable.time_windows(time_windows),
      "used vehicle": vehicle_label,
  }
  return shipment_row


def create_shipments_delivered_from_parking_location_ui():
  scenario_selector = ipywidgets.Dropdown(description="Scenario:", options=[])
  parking_selector = ipywidgets.Dropdown(description="Parking tag:", options=[])
  refresh_button = ipywidgets.Button(description="Refresh")

  display.display(
      ipywidgets.HBox(
          children=[scenario_selector, parking_selector, refresh_button]
      )
  )

  scenario = None
  parking_data = None

  def on_scenario_selected(_):
    nonlocal scenario
    nonlocal parking_data
    scenario = _scenarios[scenario_selector.value]
    parking_data = scenario.parking_location_data
    parking_selector.options = sorted(parking_data.all_parking_tags)

  scenario_selector.observe(on_scenario_selected, names=["value"])
  scenario_selector.options = _scenarios.keys()

  def on_refresh(_):
    parking_tag = parking_selector.value
    shipments = scenario.shipments
    vehicles = scenario.vehicles

    shipment_data = []
    for group_index, shipment_group in enumerate(
        parking_data.shipments_by_parking[parking_tag]
    ):
      for index_in_group, shipment_index in enumerate(shipment_group):
        row = {"index in group": index_in_group, "group": group_index}
        row.update(_shipment_row(scenario, shipment_index))
        shipment_data.append(row)

    for shipment_index in scenario.shipments_for_parking[parking_tag]:
      if shipment_index not in scenario.skipped_shipment_indices:
        continue
      row = {"index in group": "", "group": ""}
      row.update(_shipment_row(scenario, shipment_index))
      shipment_data.append(row)

    output.clear(output_tags=["table_container"])
    with output.use_tags("table_container", append=True):
      display.display(data_table.DataTable(pd.DataFrame(shipment_data)))

  refresh_button.on_click(on_refresh)


create_shipments_delivered_from_parking_location_ui()